# Exercise Homework 3

This homework will be intense. You can work on the scraping part of this homework during the TA session with Luis' help. The TA session will be there to help you - make good use of this and prepare a little before by doing step 2) before the TA session and then working on steps 3) following.

1) Get together in groups as randomized here.https://docs.google.com/spreadsheets/d/1W0qAKkJ1_J6wXDOs0PhSTnWgwA9buZmMeI3YDGIYAYY/edit?usp=sharing

2) Download the material for the homework and get the booking scrape code to run. Prove that you did by attaching a jupyter notebook. (this is only relevant for those who fail later steps)

Design and implement a mini research project in which you research the effect of a big annual event in Barcelona on rental prices on booking by scraping data for at least two separate weeks (note that search results go across different pages) for Barcelona and at least one more city.

3) Identify an event that makes a lot people come to Barcelona.

4) Think of the time periods to scrape and what second city to scrape. Explain your choices in written.

5) Write down a fixed effects regression equation that allows you to derive a difference-in-difference estimate of the effect of the event on prices. Think of controls to add, why is this relevant? Explain why you need a second city for this.

6) Scrape using a modified code that implements a loop across pages of the booking page and searches for different dates and cities.

7) Discuss two additional ideas (no need to implement): a) the use of the text that can be scraped from the hotel/appartment description b) how to de-construct the treatment effect in the effect that comes from the same hotel changing prices and composition of places changing.

### __#2. Get the booking scrape code to run.__

The code runs and prints out a list of the hotels and ratings for the dates we inputted.

### __#3. Identify an event that makes a lot people come to Barcelona.__

_Event:_ Primavera Sound is a an annual music festival held in Barcelona. The dates for this year are: Mon, May 29 – Sun, Jun 4, 2023.
    
### __#4. Think of the time periods to scrape and what second city to scrape. Explain your choices in written.__

We will scrape the rental prices for two weeks, one during the festival and the week of May 15-21, which is two weeks before the festival. We used the week two weeks before the festival instead of the week before the festival because we didn't want to accidentally capture any effect of possible increased demand for hotels during the week before the festival.  
 - Before festival: Mon, May 15 – Sun, May 21, 2023
 - During festival: Mon, May 29 – Sun, Jun 4, 2023
 
The second city we will look for is Madrid, Spain.
 
### __#5. Write down a fixed effects regression equation that allows you to derive a difference-in-difference estimate of the effect of the event on prices. Think of controls to add, why is this relevant? Explain why you need a second city for this.__

$ y = \beta_0 + \beta_1*x_1 + \beta_2*x_2 + \beta_3*(x_1-x_2) + \epsilon $

_where y is the outcome of interest, in this case the hotel prices, $x_1$ is the dummy variable for dates during the festival and $x_2$ is the dummy variable for dates before the festival._

Using the diff-in-diff estimate, we add an interaction term so that we can see the effect of the event on prices. The term $\beta_3*(x_1-x_2)$ is added to the regression equation to see estimate difference in prices from before the event to during the event.


### __#6. Scrape using a modified code that implements a loop across pages of the booking page and searches for different dates and cities.__

Using the booking_scrape code, we need to modify the code to loop across pages and search for different dates and cities. 

### __#7. Discuss two additional ideas (no need to implement):__
__a) the use of the text that can be scraped from the hotel/appartment description__

__b) how to de-construct the treatment effect in the effect that comes from the same hotel changing prices and composition of places changing.__




In [1]:
from selenium.webdriver.common.keys import Keys
import scrape_functions as kzd
import sys
import calendar
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
import re
from selenium.webdriver.support.ui import Select
import random
import time
from selenium import webdriver
import os
import numpy as np
import importlib
importlib.reload(sys.modules['scrape_functions'])
from datetime import date

In [2]:
# Starting code
geko_path = "/Users/giovannachaves/Documents/BSE/Intro to Text Mining/HW3/geckodriver.exe"

destination='/Users/giovannachaves/Documents/BSE/Intro to Text Mining/HW3/'
link = 'https://www.booking.com/'

profile, browser, download_path = kzd.start_up(destination, link, download=False)

In [3]:
# Clicking on the different buttoms to input things:

# Click on  place
place = input('Where do you want to go?')
search1 = browser.find_element('xpath','//input[@placeholder="Where are you going?"]')
search1.send_keys(place)

# Clik on dates:
kzd.check_and_click(
    browser, '//button[@class="d47738b911 e246f833f7 fe211c0731"]', type='xpath')

time.sleep(3)

dates=browser.find_elements('xpath',
    '//table[@class="aadb8ed6d3"]/tbody/tr/td/span')

from_day = input('Day from which you want to look for (In january)')
if int(from_day)< date.today().day:
    print('Wrong from_day')

to_day = input('Day until which you want to look for (In january)')


for date1 in dates:
    if date1.get_attribute("data-date") == f"2023-01-{from_day}":
        # date1.click() this is the old webdriver version
        browser.execute_script("arguments[0].click();", date1)
    if date1.get_attribute("data-date") == f"2023-01-{to_day}":
        # date1.click()
        browser.execute_script("arguments[0].click();", date1)
        break
        
kzd.check_and_click(browser, 'button.d4b6b7a9e7', type='css')
time.sleep(2)

Where do you want to go?Barcelona
Day from which you want to look for (In january)28
Day until which you want to look for (In january)31


In [4]:
# LOOPING OVER WEBPAGES:

def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath',
        '//button[text() and @class="fc63351294 f9c5690c58"]')
    total_pages = int(a[-1].text)
    return(total_pages)

In [5]:
total_pages = get_number_pages(browser)
print(f"Total pages: {total_pages}")

ratings_list=[]
hotels_list=[]

for current_page in range(total_pages):    
    hotels = browser.find_elements('xpath','//div[@class="fcab3ed991 a23c043802"]')
    ratings = browser.find_elements('xpath','//div[@class="b5cd09854e d10a6220b4"]')
    
    for i in ratings:
        ratings_list.append(i.text)
        
    for j in hotels:
        hotels_list.append(j.text)

    next_page_btn = browser.find_element('xpath', '//button[contains(@aria-label, "Next page")]')
    next_page_btn.click()
    time.sleep(10)

Total pages: 21


In [6]:
ratings_list

['8.4',
 '8.2',
 '7.7',
 '7.7',
 '7.1',
 '9.1',
 '8.7',
 '8.5',
 '8.4',
 '8.4',
 '8.3',
 '8.0',
 '8.0',
 '7.9',
 '7.9',
 '7.7',
 '7.7',
 '7.6',
 '7.6',
 '7.5',
 '7.3',
 '7.2',
 '7.1',
 '6.3',
 '6.1']

In [7]:
hotels_list

["Pars Tailor's Hostel",
 'Ciutat Vella',
 'Casa Gracia',
 'Hotel Best Auto Hogar',
 'Ant Hostel Barcelona',
 'Hostel One Ramblas',
 'Hostelle - female only hostel Barcelona',
 'Black Swan Hostel',
 'Rocket Hostels Gracia',
 '360 Hostel Barcelona Arts&Culture',
 '360 Hostel Barcelona',
 'Soul Backpackers Barcelona',
 'Hola Hostal Eixample',
 'Dream Cube Hostel',
 'Feetup Yellow Nest Hostel Barcelona',
 'Don Moustache Boutique Hostel',
 'Sun & Moon Hostel',
 'Safestay Barcelona Gothic',
 'Inout',
 'Alberguinn',
 'Mambo Tango',
 'No Limit Barcelona Central',
 'Safestay Passeig de Gràcia',
 'The Hipstel Parallel',
 'Mellow Barcelona']